In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import os

%load_ext sql

C:\Users\linru\Anaconda3\envs\hw4\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [3]:
%sql sqlite:///data/data.db

In [4]:
# test if sql loaded:

In [5]:
%%sql 
SELECT 
    token 
FROM ohlc
GROUP by 1
LIMIT 1



 * sqlite:///data/data.db
Done.


token
AAVE


In [6]:
# get tokens.

In [7]:
%%sql tokens << 
SELECT 
    token 
FROM ohlc
GROUP by 1

 * sqlite:///data/data.db
Done.
Returning data to local variable tokens


In [8]:
tokens = tokens.DataFrame()
tokens = tokens.token.tolist()

In [9]:
# get run_dates.

In [10]:
%%sql last_day << 
SELECT 
    ts
FROM ohlc
ORDER by 1 DESC
LIMIT 1

 * sqlite:///data/data.db
Done.
Returning data to local variable last_day


In [11]:
run_start = last_dow(pd.to_datetime(last_day.DataFrame().loc[0])['ts'], 0) + timedelta(days = -14)
# run_dates = [run_start + timedelta(days = x) for x in range(14)]
run_dates = [(last_dow(run_start, 0) + timedelta(days=7 * x)) for x in range(2)]

In [12]:
connection_string = 'sqlite:///data/data.db'

In [13]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [14]:
dir_output = os.getcwd()+'\\'+base_output_folder
existed_files = ['_'.join(x.split('_')[-2:]).replace('.ipynb','') for x in os.listdir(dir_output) if x.endswith('.ipynb')]


In [15]:
for run_date in run_dates:
    for token in tokens:
        if f'{token}_{(str(run_date.to_pydatetime()).split()[0])}' not in existed_files:
            print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
            res = pm.execute_notebook(
                'hw4_template.ipynb',
                f'{base_output_folder}/market_analysis_{token}_{(str(run_date.to_pydatetime()).split()[0])}.ipynb',
                # f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
                parameters = {
                    'from_date': f'{run_date}',
                    'to_date': f'{run_date + timedelta(days=7)}',
                    'token': token,
                    'connection_string': connection_string,
    #                 'events_db': events_db
                }
            )

2. Create a runner notebook using papermill to run the template notebook across all tokens in the database and the last 2 complete weeks in the database.  Summarize the following in a single DataFrame in the runner notebook
    - avg hourly return by run
    - avg volatility by run
    - maximum drawdown over the entire period of each run
    - range of last close price - first close price

In [16]:
nbs = [x for x in os.listdir(dir_output) if x.endswith('.ipynb')]
summary = pd.DataFrame({'run' : nbs})

In [17]:
def get_sum(filename):
    out_dict = {}
    nb = sb.read_notebook(dir_output + '\\' + filename)
    df = nb.scraps['prices'].data
    out_dict['avg_return'] = df.loc[:,'return'].mean()
    out_dict['avg_vol'] = df.loc[:,'volatility'].mean()
    out_dict['mdd'] = df.loc[:,'max_drawdown'].min()
    out_dict['last_minus_first'] = df.close.iloc[-1] - df.close.iloc[0]
    return out_dict

# summary.loc['
out_dict = get_sum(nbs[0])
summary.loc[:,list(out_dict.keys())] = pd.DataFrame(summary.loc[:,'run'].apply(lambda x : get_sum(x)).tolist())

In [18]:
summary

,run,avg_return,avg_vol,mdd,last_minus_first
0,market_analysis_AAVE_2021-11-29.ipynb,-0.001235,0.221054,-0.302593,-46.8080
1,market_analysis_AAVE_2021-12-06.ipynb,-0.000304,0.200400,-0.179887,-11.6730
2,market_analysis_ADA_2021-11-29.ipynb,-0.000850,0.226749,-0.250000,-0.2305
3,market_analysis_ADA_2021-12-06.ipynb,-0.000070,0.203766,-0.177991,-0.0297
4,market_analysis_ATOM_2021-11-29.ipynb,-0.000489,0.340796,-0.265123,-2.8370
5,market_analysis_ATOM_2021-12-06.ipynb,0.000112,0.254383,-0.131744,-0.0040
6,market_analysis_AVAX_2021-11-29.ipynb,-0.001428,0.320661,-0.334566,-25.6000
7,market_analysis_AVAX_2021-12-06.ipynb,0.000211,0.249173,-0.165626,1.1800
8,market_analysis_BTC_2021-11-29.ipynb,-0.000861,0.175543,-0.206462,-7979.4100
9,market_analysis_BTC_2021-12-06.ipynb,0.000127,0.134449,-0.089371,833.1100
